In [32]:
print(tf.__version__)

2.14.0


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [5]:
# Reading corpus the text file
with open("file_path_of_data", 'r', encoding='utf-8') as myfile:
    mytext = myfile.read()
mytext

'Good Morning\n\nFollowing a lavish state visit by Indian Prime Minister Narendra Modi to Washington, US President Joe Biden has called his country\'s partnership with India among the "most consequential in the world". The BBC\'s Vikas Pandey and Soutik Biswas explore the factors that contribute to the visit\'s potential in strengthening the ties between the two nations.\n\nThe US\'s relationship with India - the world\'s most populous country - is "stronger, closer and more dynamic than any time in history", Mr Biden said at the completion of a pomp-filled state visit by Mr Modi to the White House.\n\nThe remark may not be an exaggeration. "This summit suggests that the relationship has been transformed. It underscores just how broad and deep it has become in a relatively short time," says Michael Kugelman of The Wilson Center, an American think-tank.\n\nA key reason is that Washington is keen to draw India closer so that it can act as a counterbalance to China\'s growing influence in

In [6]:
mytokenizer = Tokenizer()
mytokenizer.fit_on_texts([mytext])
total_words = len(mytokenizer.word_index) + 1

In [ ]:
mytokenizer.word_index

In [9]:
my_input_sequences = []
for line in mytext.split('\n'):
    #print(line)
    token_list = mytokenizer.texts_to_sequences([line])[0]
    #print(token_list)
    for i in range(1, len(token_list)):
        my_n_gram_sequence = token_list[:i+1]
        #print(my_n_gram_sequence)
        my_input_sequences.append(my_n_gram_sequence)
        #print(input_sequences)

In [10]:
max_sequence_len = max([len(seq) for seq in my_input_sequences])
input_sequences = np.array(pad_sequences(my_input_sequences, maxlen=max_sequence_len, padding='pre'))

In [11]:
input_sequences[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0, 177, 178], dtype=int32)

In [12]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [13]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [14]:
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [15]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 82, 100)           60100     
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 601)               90751     
                                                                 
Total params: 301451 (1.15 MB)
Trainable params: 301451 (1.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
43/43 [==============================] - 10s 177ms/step - loss: 6.1866 - accuracy: 0.0430
Epoch 2/50
43/43 [==============================] - 6s 139ms/step - loss: 5.8059 - accuracy: 0.0576
Epoch 3/50
43/43 [==============================] - 7s 173ms/step - loss: 5.7235 - accuracy: 0.0576
Epoch 4/50
43/43 [==============================] - 6s 139ms/step - loss: 5.6702 - accuracy: 0.0576
Epoch 5/50
43/43 [==============================] - 9s 202ms/step - loss: 5.5883 - accuracy: 0.0576
Epoch 6/50
43/43 [==============================] - 7s 156ms/step - loss: 5.4936 - accuracy: 0.0576
Epoch 7/50
43/43 [==============================] - 7s 156ms/step - loss: 5.3807 - accuracy: 0.0641
Epoch 8/50
43/43 [==============================] - 7s 168ms/step - loss: 5.2320 - accuracy: 0.0831
Epoch 9/50
43/43 [==============================] - 6s 144ms/step - loss: 5.0419 - accuracy: 0.0911
Epoch 10/50
43/43 [==============================] - 8s 177ms/step - loss: 4.8168 - accuracy: 0.115

In [17]:
# After training the model
model.save("word_prediction.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
def Word_Prediction(input_text,predict_next_words):
  for _ in range(predict_next_words):
      token_list = mytokenizer.texts_to_sequences([input_text])[0]
      # print(token_list)
      token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
      predicted = np.argmax(model.predict(token_list), axis=-1)
      output_word = ""
      for word, index in mytokenizer.word_index.items():
          if index == predicted:
              output_word = word
              break
      input_text += " " + output_word

  return input_text

In [19]:
Word_Prediction("Good",1)

1/1 [==============================] - 0s 468ms/step


'Good morning'

In [59]:
# For the prediction you use that model multiple times

In [22]:
import tensorflow as tf
import os

In [25]:
from keras.models import load_model
model = load_model("/content/word_prediction.h5")

In [23]:
# convert keras model to tflite
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')


In [31]:
# from tensorflow.contrib import lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
file = open( 'model.tflite' , 'wb' )
file.write( model )

2.14.0


In [ ]:
# I am not able to convert the file in required file format because of some dependency with respect to the os